# Parameter Tuning

In this notebook, we will walk through an example of using BanyanArrays to
assess the effectiveness of several sets of parameters.

## Configuring

We will use Banyan to perform some data analysis on the Iris Dataset. To run this
example, please ensure that you have set up your Banyan account.

Run the first cell below to import `Banyan` and `BanyanDataFrames`.
To configure your AWS credentials, run the second cell below and provide your
AWS credentials when prompted. Banyan does not save your AWS credentials, but
they are needed so that you can run your computation in your AWS account.
Finally, run the third cell below to set your Banyan credentials and configure
Banyan.

You must pass your User ID and API Key to the `configure` function in order
to authenticate. You can find this information on the Account page of the
Banyan Dashboard. After running this cell, your credentials will be saved
in `$HOME/.banyan/banyanconfig.toml` and will be read from that file in the
future. This means that you only need to run this cell once.

In [50]:
# Import libraries
using Banyan
using BanyanArrays

# Import helper code
using("parameters.jl")

In [ ]:
# Run this cell to configure the AWS CLI. When prompted, specify your AWS
# credentials for the AWS account that you connected with Banyan. If you have
# already configured the AWS CLI with the credentials for the account you have
# configured with your Banyan account, you can skip this step.

print("Enter AWS_ACCESS_KEY_ID: \n"); sleep(1)
ENV["AWS_ACCESS_KEY_ID"] = readline()
print("Enter AWS_SECRET_ACCESS_KEY: \n"); sleep(1)
ENV["AWS_SECRET_ACCESS_KEY"] = readline()
print("Enter AWS_DEFAULT_REGION: \n"); sleep(1)
ENV["AWS_DEFAULT_REGION"] = readline()

print("AWS is now configured.")

In [ ]:
# Run this cell to configure Banyan. When prompted, provide your user ID and API
# key. You can find these on the Account page of your Banyan dashboard.
# If you have already configured Banyan, you can skip this step.

print("Please enter your User ID: \n"); sleep(1)
user_id = readline()
print("Please enter your API Key: \n"); sleep(1)
api_key = readline()

# Configures Banyan client library with your Banyan credentials
configure(user_id=user_id, api_key=api_key)
print("Banyan is now configured.")

## Creating a cluster

For this example, you will need a Banyan cluster. You can either use an existing
cluster or create a new cluster. Run the following code block and enter in either
the name of an existing cluster or the name you would like to use for a new cluster.

If you already have a cluster, you should specify its name, when prompted.

If you would like to instead create a new cluster, provide a name and the name
of the [Amazon EC2 key pair](https://docs.aws.amazon.com/AWSEC2/latest/UserGuide/ec2-key-pairs.html#having-ec2-create-your-key-pair) that you created during [Banyan setup](https://www.banyancomputing.com/creating-clusters).

In the cell below, you can change `instance_type` to create a cluster with a
different EC2 instance type that may have a larger amount of memory or workers.
See the documentation [here](https://www.banyancomputing.com/banyan-jl-docs/create-cluster/) for the other parameters for creating a cluster.

In [51]:
print("Cluster name for existing cluster or new cluster: \n"); sleep(1)
cluster_name = readline()
println(cluster_name)

clusters = get_clusters()
println("You have $(length(clusters)) clusters")
if !(haskey(clusters, cluster_name) && clusters[cluster_name].status == :running)
    println("Creating new cluster $(cluster_name)")
    print("Name of SSH EC2 Key Pair: \n"); sleep(1)
    ec2_key_pair_name = readline()
    println(ec2_key_pair_name)
    create_cluster(
        name=cluster_name,
        instance_type="t3.2xlarge",
        initial_num_workers=2,
        ec2_key_pair_name=ec2_key_pair_name
    )
else
    println("Using existing cluster $(cluster_name)")
end
get_cluster(cluster_name)

Cluster name for existing cluster or new cluster: 
image-processing
You have 32 clusters
Creating new cluster image-processing
Name of SSH EC2 Key Pair: 
CailinSSHKeyPair


┌ Info: Started creating cluster named image-processing
└ @ Banyan c:\Users\caili\Projects\Banyan\banyan-julia\Banyan\src\clusters.jl:84
✓ Setting up cluster image-processing 	 Time: 0:26:21


Cluster("image-processing", :running, "", 0, "arn:aws:s3:::banyan-cluster-data-image-processing-53ac0d37")

## Start a session

In order to perform any computation, we need to allocate resources on the cluster
to perform the computation. To do this, start a session with a specified number
of workers. The number of workers is correlated to the amount of parallelism
and speedup you can get. The more workers, the more parallelized your computation
can be and potentially the faster it can run. For this example, 2 workers should
be sufficient.

In [ ]:
# Start a session
session_id = start_session(
    cluster_name = cluster_name,
    nworkers = 10,
    code_files = ["parameters.jl"],
    force_update_files = true,
    print_logs = false,  # Toggle this if you want to view job output printed here in the notebook
    store_logs_in_s3 = true,  # Toggle this if you do not want job logs to be saved
)

## Construct an array of structs

**TODO: add more details here.**

In [ ]:
configurations = [
    Configuration(
        rand(5:150),
        rand(5:150)
    )
    for i=1:100
]

## Cleanup resources

After running your desired computation, we suggest that you end your running
session, so that you are not charged for resources when you are not using them. You
may choose to keep your cluster running.

Run the following code block to end the session that you created in this example.

In [ ]:
# End session
end_session(session_id)